# Notebook 01: basic data preparation (XLSX >>> cleaning >>> CSV)

Authors: Fabio Morea, Leyla Vesnic @ Area Science Park
 
Description: python scripts to clean and prepare data on regional companies.
This script imports data from 'IMPRESE.xlsx' and produces clean data in the form of .csv files.

License: 
scripts are available under CC-BY-4.0
data is not included in the package

Code is mantained on GitLab: https://gitlab.com/area-science-park-sustainability/imprese-fvg

# comments and instructions for developers
i dati sono contenuti in 2 fogli di lavoro
vengono convertiti in 2 file csv 

per approfondire: trattare i csv è più veloce rispetto a xls e consente condivisione nel data repository) cfr: https://stackoverflow.com/questions/50695778/how-to-increase-process-speed-using-read-excel-in-pandas

* creazione id_sede (corrisponde a CF, ma 
                        è più efficiente 
                        non è "dato personale")

* creazione id_impresa (un'impresa ha più sedi identificate da un numero 0 = SEDE , 1 = UL1...)

directory o file di input e output:
        ....

operazioni preliminari:
    1- spostare il file IMPRESE.xlsx nella directory di lavoro

operazioni eseguite da questo script

In [114]:
# Setup
import sys
import os
from pathlib import Path
import datetime
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

In [115]:
current_path = Path(os.getcwd())
data_subdir = "data"
data_path = current_path.parent / data_subdir
data_dir = str(data_path)
listafile = os.listdir(data_path)
listafile = list(filter(lambda x: 'imprese_fvg_' in x, listafile))  
assert len(listafile) >= 1, f"Errore: non trovo i file dati nella cartella {data_dir}"

In [116]:
file_da_elaborare = '10_2023'   # mese da modificare ad ogni aggiornamento
excel_file  = data_dir + '\\imprese_fvg_' + file_da_elaborare + '.xlsx'
print(f'Carico i dati presenti nel file {excel_file}')

Carico i dati presenti nel file c:\Users\longato\OneDrive - Area Science Park\Data Repository\dataset_imprese_fvg\data\imprese_fvg_10_2023.xlsx


In [117]:
xl = pd.ExcelFile(  excel_file, engine="openpyxl")
xl_sheets = xl.sheet_names  # see all sheet names
assert xl_sheets == ['FRIULI Anagrafica', 'FRIULI codice attività']

C:\Users\longato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Primo foglio: FRIULI Anagrafica

In [118]:
df_anagrafica = xl.parse(   'FRIULI Anagrafica', 
                            header = 0, 
                            dtype=str,
                            keep_default_na=False)

C:\Users\longato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell S64960 is marked as a date but the serial value 3032437.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
C:\Users\longato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell S64961 is marked as a date but the serial value 3032437.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [119]:
#crea dizionario con i nomi di colonna originali e corretti
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='anagrafica') 
l1 = cols_df['nomi_colonne_originali']
l2 = cols_df['nomi_colonne_corretti']
cols_dic = dict(zip(l1,l2))
df_anagrafica.rename(columns=cols_dic, inplace=True)

In [120]:
df_anagrafica['fonte'] = 'I'
df_anagrafica['mm_aaaa'] = file_da_elaborare

In [121]:
df_anagrafica['n_sede'] = df_anagrafica['sede_ul'].str[3:] 
df_anagrafica.loc[ df_anagrafica['n_sede'] == 'E', 'n_sede'] = '0'
#alcune UL hanno separatore delle migliaia!
df_anagrafica['n_sede'] = df_anagrafica['n_sede'].str.replace('.' ,'', regex = True) 

In [122]:
# TODO: aggiungere controlli sui dati mancanti: sostituire con NaN o con "" 
dim = df_anagrafica.shape #numero di righe e colonne del dataset
print(f'dimensioni del dataset (righe,colonne) = {dim}\n')
print(df_anagrafica.info())

dimensioni del dataset (righe,colonne) = (199374, 58)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199374 entries, 0 to 199373
Data columns (total 58 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   cf                     199374 non-null  object
 1   prov                   199374 non-null  object
 2   reg_imp_n              199374 non-null  object
 3   rea                    199374 non-null  object
 4   sede_ul                199374 non-null  object
 5   n-albo_art             199374 non-null  object
 6   reg_imp_sez            199374 non-null  object
 7   ng2                    199374 non-null  object
 8   ng_esteso              199374 non-null  object
 9   tipo_impresa           199374 non-null  object
 10  data_isc_ri            199374 non-null  object
 11  data_isc_rd            199374 non-null  object
 12  data_isc_aa            199374 non-null  object
 13  data_apert_ul          199374 non-null  object
 1

In [123]:
#strip special characters
# TODO da migliorare con re
#chars_to_strip = '\\n\\t\\r|#'
cols_to_strip = [ 'denominazione',  'descrizione_attivita', 'indirizzo']
for ccc in cols_to_strip:
        if isinstance(ccc, str): 
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\n',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\t',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\\r',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '"' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  "'" ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '|' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '_x000D_', ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '#' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '#' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '\*+',     ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.replace(  '-' ,      ' ', regex = True)
                df_anagrafica[ccc] = df_anagrafica[ccc].str.strip()
                print(ccc, " ok")

df_anagrafica['reg_imp_n'] = df_anagrafica['reg_imp_n'].str.replace(r'\(', '', regex=True)
df_anagrafica['reg_imp_n'] = df_anagrafica['reg_imp_n'].str.replace(r'\)', '', regex=True)


denominazione  ok
descrizione_attivita  ok
indirizzo  ok


In [124]:
df_anagrafica= df_anagrafica.loc[df_anagrafica['data_cess_att'].isin([''])]

In [125]:
# Le date nel file originale hanno alcuni problemi: (1) sottrarre 3000 all'anno, 
# (2) gestire correttamente errori e "nan", (3)eliminare date antecedenti al 1800 o successive al 2099

def anno_corretto(dstring:str) -> str:
    num =0
    dstring = str(dstring)[:10]
    if len(dstring) < 4:
        result = "NaT"
    else:
        try:
            num = int(dstring[0:2])
            if num >= 48 and num <= 51: #anni fra il 1800 e il 2100
                num = num - 30
                result = str(num) + dstring[2:]
            else:
                result = "NaT"
        except:
            result = "NaT"
    return result   

In [126]:
assert anno_corretto('x')   == 'NaT'
assert anno_corretto('1799-03-01') == 'NaT'
assert anno_corretto('2100-03-01') == 'NaT'
assert anno_corretto('4987-03-01') == '1987-03-01'
assert anno_corretto('5021-03-01') == '2021-03-01'
pass

In [127]:
cols_date = ['data_isc_ri', 'data_isc_aa', 'data_liquid','data_cess_att',  'data_fall', 'data_apert_ul', 'data_fine_aa']
#'data_isc_rd'
#'data_ini_at'
#'data_cost'

# i tre campi data in formato commento non vengono gestiti dalla funzione "anno corretto" in modo da lasciare il +3000 da poter gesitre in excel

#cols_date = [d for d in df_anagrafica.columns if d.startswith('data')]
print(cols_date)
for col in cols_date:
    datestring3000 = df_anagrafica[col].tolist()
    #print(datestring3000)
    datestring = [anno_corretto(item) for item in datestring3000]
    df_anagrafica[col] = pd.to_datetime(datestring)#, errors = 'coerce')
    #print(df_anagrafica[col])

['data_isc_ri', 'data_isc_aa', 'data_liquid', 'data_cess_att', 'data_fall', 'data_apert_ul', 'data_fine_aa']


### id localiz e id impresa

In [128]:
#creo key_clf che servirà per connettere id_localiz al dataset dei codici
df_anagrafica['key_cfl'] = df_anagrafica['cf'] + '_'  + df_anagrafica['n_sede'].apply(str)
df_anagrafica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195917 entries, 0 to 199373
Data columns (total 59 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   cf                     195917 non-null  object        
 1   prov                   195917 non-null  object        
 2   reg_imp_n              195917 non-null  object        
 3   rea                    195917 non-null  object        
 4   sede_ul                195917 non-null  object        
 5   n-albo_art             195917 non-null  object        
 6   reg_imp_sez            195917 non-null  object        
 7   ng2                    195917 non-null  object        
 8   ng_esteso              195917 non-null  object        
 9   tipo_impresa           195917 non-null  object        
 10  data_isc_ri            194283 non-null  datetime64[ns]
 11  data_isc_rd            195917 non-null  object        
 12  data_isc_aa            31259 non-null   datetime6

In [129]:
# creo colonna temporanea anni per ordinare gli indici
df_anagrafica['min_date']   = pd.to_datetime( df_anagrafica[cols_date].min(axis=1) )
df_anagrafica['today']      = pd.to_datetime( pd.Timestamp.today().strftime('%Y-%m-%d') ) 
df_anagrafica['anni_timedelta'] =  (df_anagrafica['today']  - df_anagrafica['min_date'])
df_anagrafica['anni'] = df_anagrafica['anni_timedelta'].dt.total_seconds() / (60 * 60 * 24 * 365)
df_anagrafica.drop(columns='today')
df_anagrafica.drop(columns='anni_timedelta')

,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,stato_impresa,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni
0,00000470310,GO,GO007-1352,37843,SEDE,,O,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,1996-02-19,4975-01-14 00:00:00,NaT,NaT,,,NaT,NaT,NaT,P E L L I Z Z A R I S I L V I O D I S E ...,V I A P E S C H E R I A 4,,34071,CORMONS - GO,,,1999,0,0,00000470310,0481/60323,,C O M M E R C I O A L M I N U T O T A B ...,,INATTIVA,,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,,NaT,4974-08-26 00:00:00,SE - SEDE PRINCIPALE,2022-08-01 00:00:00,GO,37843,0,I,10_2023,,00000470310_,1996-02-19,2023-10-17,27.676712
1,00002070324,TS,TS006-7084,65026,SEDE,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4969-01-30 00:00:00,NaT,NaT,,4969-01-30 00:00:00,NaT,NaT,NaT,B . F . B . C A S A D I S P E D I Z ...,V I A C O R T I 2,,34123,TRIESTE - TS,,,2023,0,16,00002070324,040/3220798,20000,"S P E D I Z I O N I D O G A N A L I , D A ...",EURO,ATTIVA,,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,NaT,4969-01-30 00:00:00,SE - SEDE PRINCIPALE,2023-07-28 00:00:00,TS,65026,0,I,10_2023,,00002070324_,1996-02-19,2023-10-17,27.676712
2,00002070324,TS,TS006-7084,65026,UL-2,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4969-01-30 00:00:00,NaT,2007-08-01,,,NaT,NaT,NaT,B . F . B . C A S A D I S P E D I Z ...,F E R N E T T I 5,,34016,MONRUPINO - TS,,"AUTOPORTO, MODULO 50",2023,0,16,00002070324,040/3220798,20000,"S P E D I Z I O N I D O G A N A L I , D A ...",EURO,ATTIVA,U - UFFICIO,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,NaT,4969-01-30 00:00:00,UL - UNITÀ LOCALE,2023-07-28 00:00:00,TS,65026,2,I,10_2023,,00002070324_,1996-02-19,2023-10-17,27.676712
3,00002070324,TS,TS006-7084,65026,UL-4,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4969-01-30 00:00:00,NaT,2015-10-15,,,NaT,NaT,NaT,B . F . B . C A S A D I S P E D I Z ...,P U N T O F R A N C O N U O V O E X ...,,34123,TRIESTE - TS,,,2023,0,16,00002070324,040/3220798,20000,"S P E D I Z I O N I D O G A N A L I , D A ...",EURO,ATTIVA,U - UFFICIO,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,CLAUDIO.BROSCH@PEC.BFBTRIESTE.COM,NaT,4969-01-30 00:00:00,UL - UNITÀ LOCALE,2023-07-28 00:00:00,TS,65026,4,I,10_2023,,00002070324_,1996-02-19,2023-10-17,27.676712
5,00006350938,PN,PN033-1275,9764,SEDE,,O,AS,AS - SOCIETA' IN ACCOMANDITA SEMPLICE,SOCIETA' DI PERSONE,1996-02-19,4965-11-18 00:00:00,NaT,NaT,,4965-10-30 00:00:00,NaT,NaT,NaT,C A R R O Z Z E R I A A Z Z A N E S E D I ...,V I A F I U M I C I N O 1 4,,33082,AZZANO DECIMO - PN,,,2023,1,0,00006350938,0434/631194,0,D A L 0 9 / 0 7 / 2 0 1 3 C O M M E R C I ...,EURO,ATTIVA,,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,CAR.AZZANESESNC@TICERTIFICA.IT,NaT,4965-10-30 00:00:00,SE - SEDE PRINCIPALE,2023-02-03 00:00:00,PN,9764,0,I,10_2023,,00006350938_,1996-02-19,2023-10-17,27.676712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199369,ZZZNTN78H14L195O,UD,UD-2009-7369,270774,SEDE,UD-87280,A-P,DI,DI - IMPRESA INDIVIDUALE,IMPRESE INDIVIDUALI,2009-03-10,5009-03-10 00:00:00,2009-03-10,NaT,,5009-03-10 00:00:00,NaT,NaT,

In [130]:
df_anagrafica.sort_values(by = ['anni', 'key_cfl'], ascending=False, inplace = True)

In [131]:
# creo id_localiz (dall'indice, mi aspetto 205646)
df_anagrafica.reset_index(inplace=True)
df_anagrafica.reset_index(inplace=True)
df_anagrafica['id_localiz'] = df_anagrafica['index'] + 1

df_anagrafica.sort_values(by = 'index', inplace = True)

In [132]:
#creo id impresa univoco collegato a CF
df_cf_univoco = df_anagrafica[['cf', 'denominazione']]
df_cf_univoco.drop_duplicates(subset='cf', inplace = True)
df_cf_univoco.reset_index(inplace = True)
df_cf_univoco.reset_index(inplace = True)
df_cf_univoco['id_impresa'] = df_cf_univoco['level_0'] +1
df_cf_univoco.columns

C:\Users\longato\AppData\Local\Temp\ipykernel_5100\3926984543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf_univoco.drop_duplicates(subset='cf', inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_5100\3926984543.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf_univoco['id_impresa'] = df_cf_univoco['level_0'] +1


Index(['level_0', 'index', 'cf', 'denominazione', 'id_impresa'], dtype='object')

In [133]:
df_cf_univoco.drop(columns=['level_0', 'index'], inplace = True)
cols_order = ['id_impresa', 'cf' ]
df_cf_univoco = df_cf_univoco[cols_order]

C:\Users\longato\AppData\Local\Temp\ipykernel_5100\2642237389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf_univoco.drop(columns=['level_0', 'index'], inplace = True)


In [134]:
df_anagrafica.shape[0]

195917

In [135]:
df_anagrafica = df_anagrafica.merge(df_cf_univoco, on = 'cf', how = 'inner') 

In [136]:
df_anagrafica.shape[0]

195917

### Controlli su unità locali senza sedi di riferimento - controllo duplicati - controllo date

In [137]:
# funzione per creare nuova colonna dicotomica
def f(row):
    if row['sede_ul'] == 'SEDE':
        val = 's'
    else:
        val = 'u'
    return val

In [138]:
# nuova colonna 
df_anagrafica['SEDE_UL_num'] = df_anagrafica.apply(f, axis = 1)

In [139]:
# tabella pivot - conteggio sedi/unità locali per cf
pivot_test = df_anagrafica.pivot_table(index = "cf",
                                values = "sede_ul",
                                columns='SEDE_UL_num',
                                aggfunc = lambda x : x.count(),
                                margins = True,
                                fill_value=0)
pivot_test

SEDE_UL_num,s,u,All
cf,,,
00000470310,1,0,1
00002070324,1,2,3
00006350938,1,0,1
00006500938,1,0,1
00006550313,1,0,1
...,...,...,...
ZZZRRT70L67L424I,1,0,1
ZZZRRT74D06Z700T,1,0,1
ZZZSVN46E60G381I,1,0,1


In [140]:
#lista cf senza sede
df_test = pivot_test
cf_no_sede = df_test[ df_test['s']==0 ]
cf_no_sede = cf_no_sede.reset_index()
cf_no_sede = cf_no_sede['cf'].to_list()


In [141]:
df_anagrafica = df_anagrafica[~df_anagrafica['cf'].isin(cf_no_sede)]

In [142]:
df_anagrafica.groupby('stato_impresa').count()

,level_0,index,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni_timedelta,anni,id_localiz,id_impresa,SEDE_UL_num
stato_impresa,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ATTIVA,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,178462,180044,31169,86554,180044,180044,0,1593,182,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,0,180044,180044,180044,180044,180044,180044,180044,180044,180044,180044,179027,180044,179027,179027,180044,180044,180044
CN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
FALLIMENTO,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2577,2578,11,1944,2578,2578,0,2275,360,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,0,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578,2578
INATTIVA,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10396,10442,14,1745,10442,10442,0,3630,1312,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,0,10442,10442,10442,10442,10442,10442,10442,10442,10442,10442,10415,10442,10415,10415,10442,10442,10442
LIQUIDAZIONE,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2406,2409,51,618,2409,2409,0,357,1903,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,0,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409,2409
LT,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,1,5,5,0,0,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
LZ,65,65,65,65,65,65,65,65,65,65,65,65,65,65,2,39,65,65,0,0,19,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,0,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65
PU,25,25,25,25,25,25,25,25,25,25,25,25,25,25,3,10,25,25,0,0,1,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,0,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
RA,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,5,7,7,0,1,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


### Gestione dei duplicati

In [143]:
#df_anagrafica['id_unitàlocale'] = df_anagrafica['cf'] + df_anagrafica['sede_ul'] + df_anagrafica['comune']
#df_anag_dup = df_anagrafica[df_anagrafica[['id_unitàlocale']].duplicated()]
#df_anag_dup = df_anag_dup[~df_anag_dup['stato_impresa'].isin(['ATTIVA'])]
#id_dup = df_anag_dup.index.values.tolist()
#df_anagrafica.drop(id_dup, axis=0, inplace=True)
#df_anag_dup.tail(20)

In [144]:
#df_2 = df_anagrafica[df_anagrafica[['id_unitàlocale']].duplicated(keep='last')]
#id_dup_2 = df_2.index.values.tolist()
#df_anagrafica.drop(id_dup_2, axis=0, inplace=True)

In [145]:
df_dup = df_anagrafica[df_anagrafica.duplicated(subset=['cf','sede_ul'], keep=False)]

In [146]:
df_dup = df_dup[df_dup['prov'] != df_dup['prov_camera_commercio']]
df_dup[df_dup['cf'] == '07381630966']

,level_0,index,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,stato_impresa,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni_timedelta,anni,id_localiz,id_impresa,SEDE_UL_num
115678,130159,118581,07381630966,UD,MI-2011-59418,1954900,UL-4,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,2011-03-22,5011-03-22 00:00:00,NaT,2014-04-28,,,NaT,NaT,NaT,G I O I A G I O C H I S . R . L .,V I A P A L M A N O V A 8 7,,33100,UDINE - UD,,,2023,0,13,07381630966,,20000,,EURO,ATTIVA,SAD - SALA GIOCHI,,,,,,,NO,NO,FORTE,NON GIOVANE,ITALIANA,GIOIAGIOCHI@CERTAPEC.IT,NaT,5011-03-16 00:00:00,UL - UNITÀ LOCALE,2023-02-18 00:00:00,MI,1954900,4,I,10_2023,,07381630966_,2011-03-22,2023-10-17,4592 days,12.580822,118582,48992,u
115680,130161,118583,07381630966,UD,MI-2011-59418,1954900,UL-5,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,2011-03-22,5011-03-22 00:00:00,NaT,2015-10-17,,,NaT,NaT,NaT,G I O I A G I O C H I S . R . L .,V I A L E P A L M A N O V A 8 9,,33100,UDINE - UD,,,2023,0,13,07381630966,,20000,,EURO,ATTIVA,BAR - BAR,,,,,,,NO,NO,FORTE,NON GIOVANE,ITALIANA,GIOIAGIOCHI@CERTAPEC.IT,NaT,5011-03-16 00:00:00,UL - UNITÀ LOCALE,2023-02-18 00:00:00,MI,1954900,5,I,10_2023,,07381630966_,2011-03-22,2023-10-17,4592 days,12.580822,118584,48992,u


In [147]:
id_df_dup = df_dup.index.values.tolist()
df_anagrafica.drop(id_df_dup, axis=0, inplace=True)

In [148]:
df_dup_test_final = df_anagrafica[df_anagrafica.duplicated(subset=['cf','sede_ul'], keep=False)]
df_dup_test_final

,level_0,index,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,stato_impresa,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni_timedelta,anni,id_localiz,id_impresa,SEDE_UL_num
258,1942,268,00050830314,GO,TV-2003-39672,301147,UL-1,,G-O,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,2003-08-06,5003-08-05 00:00:00,NaT,1980-12-22,,,NaT,NaT,NaT,S O C I E T A A G R I C O L A T U R C O ...,L O C A L I T A D O S S I B O S C A T,,34073,GRADO - GO,,,2023,0,1,01001730306,,0,A T T I V I T A : S V O L T A P R E S S ...,EURO,ATTIVA,AA - AZIENDA AGRICOLA,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,AGRICOLA.TURCO@PEC.IT,NaT,4980-12-22 00:00:00,UL - UNITÀ LOCALE,2021-07-14 00:00:00,GO,61448,1,I,10_2023,,00050830314_,1980-12-22,2023-10-17,15639 days,42.846575,269,155,u
261,93312,271,00050830314,UD,TV-2003-39672,301147,UL-1,,G-O,SN,SN - SOCIETA' IN NOME COLLETTIVO,SOCIETA' DI PERSONE,2003-08-06,5003-08-05 00:00:00,NaT,2018-12-17,,,NaT,NaT,NaT,S O C I E T A A G R I C O L A T U R C O ...,V I A T A L M A S S O N S 6,,33061,RIVIGNANO TEOR - UD,,,2023,0,1,01001730306,,0,A T T I V I T A : S V O L T A P R E S S ...,EURO,ATTIVA,AA - AZIENDA AGRICOLA,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,AGRICOLA.TURCO@PEC.IT,NaT,4980-12-22 00:00:00,UL - UNITÀ LOCALE,2021-07-14 00:00:00,UD,350772,1,I,10_2023,,00050830314_,2003-08-06,2023-10-17,7377 days,20.210959,272,155,u
786,42828,813,00065030256,TS,BL006-1564,37497,UL-1,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4966-07-06 00:00:00,NaT,2021-05-01,,,NaT,NaT,NaT,P A S S U E L L O F . L L I S . R . L .,V I A C O M B I 2 2 / A,,34143,TRIESTE - TS,,,2023,0,57,00065030256,0435/32441,156000,D A L 0 1 . 0 1 . 1 9 9 9 C O M M E R C I ...,EURO,ATTIVA,U - UFFICIO,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,AMMINISTRAZIONE@PEC.PASSUELLOFRATELLI.IT,NaT,4966-06-14 00:00:00,UL - UNITÀ LOCALE,2023-06-07 00:00:00,TS,208312,1,I,10_2023,,00065030256_,1996-02-19,2023-10-17,10102 days,27.676712,814,317,u
787,42829,814,00065030256,UD,BL006-1564,37497,UL-1,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4966-07-06 00:00:00,NaT,2023-06-01,,,NaT,NaT,NaT,P A S S U E L L O F . L L I S . R . L .,P I A Z Z A G I U S E P P E M A Z Z I N I 9,,33028,TOLMEZZO - UD,,,2023,0,57,00065030256,0435/32441,156000,D A L 0 1 . 0 1 . 1 9 9 9 C O M M E R C I ...,EURO,ATTIVA,U - UFFICIO,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,AMMINISTRAZIONE@PEC.PASSUELLOFRATELLI.IT,NaT,4966-06-14 00:00:00,UL - UNITÀ LOCALE,2023-06-07 00:00:00,UD,368668,1,I,10_2023,,00065030256_,1996-02-19,2023-10-17,10102 days,27.676712,815,317,u
1480,42316,1528,00093830792,UD,VV047-955,57593,UL-1,,O,SR,SR - SOCIETA' A RESPONSABILITA' LIMITATA,SOCIETA' DI CAPITALE,1996-02-19,4966-03-22 00:00:00,NaT,2006-11-17,,,NaT,NaT,NaT,D I S T I L L E R I A F . L L I C A F F O ...,V I A S A N D A N I E L E 9 / 1 1,,33037,PASIAN DI PRATO - UD,PASSON,,2023,0,43,00093830792,0963/85025,1000000,"P R O D U Z I O N E D I L I Q U O R I , ...",EURO,ATTIVA,DIS - DISTILLERIA,,,,,,,NO,NO,NON FEMMINILE,NON GIOVANE,ITALIANA,INFO@PEC.CAFFO.COM,NaT,4965-07-15 00:00:00,UL - UNITÀ LOCALE,2023-07-09 00:00:00,UD,262361,1,I,10_2023,,00093830792_,1996-02-19,2023-10-17,10102 days,27.676712,1529,651,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [149]:
# verifico che fra i duplicati rimasti non ci siano le stesse ul nella stessa provincia
df_dup_test_final_2 = df_dup_test_final[df_dup_test_final.duplicated(subset=['cf','sede_ul','prov'], keep=False)]
df_dup_test_final_2

,level_0,index,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,stato_impresa,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni_timedelta,anni,id_localiz,id_impresa,SEDE_UL_num


In [150]:
#f_3 = df_anagrafica[df_anagrafica.duplicated(subset=['cf','sede_ul'], keep=False)]
#df_3 = df_3[~df_3['stato_impresa'].isin(['ATTIVA'])]
#id_df_3 = df_3.index.values.tolist()
#df_anagrafica.drop(id_df_3, axis=0, inplace=True)

In [151]:
# df_4 = df_anagrafica[df_anagrafica.duplicated(subset=['cf','sede_ul'], keep='last')]
# id_df_4 = df_4.index.values.tolist()
# df_anagrafica.drop(id_df_4, axis=0, inplace=True)

In [152]:
#df_anagrafica = df_anagrafica.drop('id_unitàlocale', axis=1)

In [153]:
df_anagrafica.groupby('stato_impresa').count()

,level_0,index,cf,prov,reg_imp_n,rea,sede_ul,n-albo_art,reg_imp_sez,ng2,ng_esteso,tipo_impresa,data_isc_ri,data_isc_rd,data_isc_aa,data_apert_ul,data_canc,data_ini_at,data_cess_att,data_fall,data_liquid,denominazione,indirizzo,indirizzo_strad,indirizzo_cap,comune,indirizzo_fraz,indirizzo_altre,addetti_aaaa,addetti_indip,addetti_dip,piva,tel,capitale,descrizione_attivita,capitale_valuta,tipo_sedeul_1,tipo_sedeul_2,tipo_sedeul_3,tipo_sedeul_4,tipo_sedeul_5,imp_sedi_ee,imp_eefvg,imp_pmi,imp_startup,imp_femmilile,imp_giovanile,imp_straniera,PEC,data_fine_aa,data_cost,tipo_localizzazione,last_update,prov_camera_commercio,REA_camera_commercio,PRO_LOCALIZZAZIONE,fonte,mm_aaaa,n_sede,key_cfl,min_date,today,anni_timedelta,anni,id_localiz,id_impresa,SEDE_UL_num
stato_impresa,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ATTIVA,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,176391,177967,31158,84520,177967,177967,0,1471,182,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,0,177967,177967,177967,177967,177967,177967,177967,177967,177967,177967,176950,177967,176950,176950,177967,177967,177967
CN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
FALLIMENTO,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2419,2420,11,1789,2420,2420,0,2117,351,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,0,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420,2420
INATTIVA,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10296,10342,14,1648,10342,10342,0,3574,1290,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,0,10342,10342,10342,10342,10342,10342,10342,10342,10342,10342,10315,10342,10315,10315,10342,10342,10342
LIQUIDAZIONE,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2353,2356,51,565,2356,2356,0,345,1875,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,0,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356,2356
LT,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,1,5,5,0,0,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
LZ,64,64,64,64,64,64,64,64,64,64,64,64,64,64,2,38,64,64,0,0,18,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,0,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
PU,24,24,24,24,24,24,24,24,24,24,24,24,24,24,3,9,24,24,0,0,1,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,0,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
RA,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,5,7,7,0,1,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [154]:
df_anagrafica.shape[0]

193527

## Secondo foglio: FRIULI codici attività

In [155]:
df_codici = xl.parse('FRIULI codice attività',  
                    header = 0,
                    dtype=str,
                    keep_default_na=False) 

In [156]:
#crea dizionario con i nomi di colonna originali e corretti
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='codici') 
l1 = cols_df['nomi_colonne_originali']
l2 = cols_df['nomi_colonne_corretti']
cols_dic = dict(zip(l1,l2))

df_codici.rename(columns=cols_dic, inplace=True)

In [157]:
df_codici['fonte'] = 'I'
df_codici['mm_aaaa'] = file_da_elaborare

In [158]:
# creo indice key_clf anche nel df_codici per poter collegare id_localiz
df_codici['key_cfl'] = df_codici['cf'] + '_'  + df_codici['loc_n']

df_temp = df_anagrafica[['key_cfl',  'id_impresa', 'id_localiz']]

# aggiungo id_localiz in df_codici
df_codici = df_codici.merge(df_temp, on = 'key_cfl', how = 'inner') 

In [159]:
# filtriamo anche il foglio codici eliminando i cf senza sede
df_codici = df_codici[~df_codici['cf'].isin(cf_no_sede)]

In [160]:
df_cod_dup = df_codici[df_codici.duplicated(subset=['cf','prov', 'rea', 'loc_n', 'ateco_tipo','ateco','ateco_desc'], keep=False)]
df_cod_dup

,cf,prov,rea,loc_n,ateco_tipo,ateco,ateco_desc,fonte,mm_aaaa,key_cfl,id_impresa,id_localiz


In [161]:
df_codici.shape[0]

0

### Salvataggio di file .csv per diverse destinazioni

#### File .csv per Repository

Dovendo fondere i campi relativi a "tipo_sedeul_n", creo una copia del dataframe per poter operare la fusione sulla copia e salvarla nella versione "Repository"

In [162]:
df_anagrafica_repo = df_anagrafica.copy()
# unione dei campi tipo_sedeul
df_anagrafica_repo['tipo_sedeul'] =  df_anagrafica_repo.tipo_sedeul_1+df_anagrafica_repo.tipo_sedeul_2+df_anagrafica_repo.tipo_sedeul_3+df_anagrafica_repo.tipo_sedeul_4+df_anagrafica_repo.tipo_sedeul_5
df_anagrafica_repo.drop(columns=['tipo_sedeul_1', 'tipo_sedeul_2', 'tipo_sedeul_3', 'tipo_sedeul_4',
       'tipo_sedeul_5'], inplace=True)

In [163]:
# Salva il file ANAGRAFICA in versione "repository": 
file_risultati = data_dir + '\\' + 'imprese_anagrafica.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='anagrafica',
            usecols = ['nomi_colonne_export','ordine_repo']).dropna() 
cols_df.sort_values('ordine_repo', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_export'])
df_anagrafica_repo[cols_to_use].to_csv(  file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

In [ ]:
# Salva il file CODICI in versione "repository": 
file_risultati = data_dir + '\\' + 'imprese_codici.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='codici',
            usecols = ['nomi_colonne_corretti','ordine_repo']).dropna() 
cols_df.sort_values('ordine_repo', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_codici[cols_to_use].to_csv(      file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

#### File. csv per Innvoation Intelligence (unità locali anche extra FVG)

In [ ]:
# Salva il file ANAGRAFICA file in versione "innovation intelligence": 
file_risultati = data_dir + '\\' + 'iifvg_anagrafica.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='anagrafica',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_anagrafica[cols_to_use].to_csv(  file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

In [ ]:
# Salva il file CODICI in versione "innovation intelligence": 
file_risultati = data_dir + '\\' + 'iifvg_codici.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='codici',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_codici[cols_to_use].to_csv(      file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

#### File .csv per Innovation Intelligence (solamente unità locali FVG)

In [ ]:
local_sede = ["SEDE"]
prov_FVG = ["GO", "PN", "UD", "TS"]
#filtro ed elimino le unità locali extra FVG presenti nell'anagrafica di Insiel
unità_locali_extraFVG_filter = ~df_anagrafica["sede_ul"].isin(local_sede) & ~df_anagrafica["prov"].isin(prov_FVG)
#print(~unità_locali_extraFVG_filter)
#df_anagrafica[unità_locali_extraFVG_filter].drop()
df_anagrafica = df_anagrafica[~unità_locali_extraFVG_filter]

In [ ]:
# Salva il file ANAGRAFICA file in versione "innovation intelligence" senza UL extra FVG
file_risultati = data_dir + '\\' + 'iifvg_anagrafica_filtrato.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='anagrafica',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_anagrafica[cols_to_use].to_csv(  file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

In [ ]:
local_sede = ["0"]
prov_FVG = ["GO", "PN", "UD", "TS"]
#filtro ed elimino le unità locali extra FVG presenti nel folgio codici dell'anagrafica Insiel
unità_locali_extraFVG_filter = ~df_codici["loc_n"].isin(local_sede) & ~df_codici["prov"].isin(prov_FVG)
#print(~unità_locali_extraFVG_filter)
#df_codici[unità_locali_extraFVG_filter].drop()
df_codici = df_codici[~unità_locali_extraFVG_filter]

In [ ]:
# Salva il file CODICI in versione "innovation intelligence" senza UL extra FVG
file_risultati = data_dir + '\\' + 'iifvg_codici_filtrato.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='codici',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_codici[cols_to_use].to_csv(      file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

#### File .csv per Innovation Intelligence (SOLO unità locali FVG e SOLO società NON di capitale) - per integrazione nella piattaforma di I2

In [ ]:
tipo_società = ["SOCIETA' DI CAPITALE"]
#filtro ed elimino le società di capitale
anagrafica_soc_capitale = df_anagrafica["tipo_impresa"].isin(tipo_società)
df_anagrafica = df_anagrafica[~anagrafica_soc_capitale]

In [ ]:
# Salva il file ANAGRAFICA in versione "innovation intelligence" per integrazione dati
file_risultati = data_dir + '\\' + 'iifvg_anagrafica_filtrato_noncap.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='anagrafica',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_anagrafica[cols_to_use].to_csv(  file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)

In [ ]:
# filtro del foglio CODICI in base ai cf di df_anagrafica (ultima versione)
filtro_df_anagrafica = df_anagrafica[['cf']].copy()
df_codici = df_codici[df_codici['cf'].isin(filtro_df_anagrafica['cf'].values)]

In [ ]:
# Salva il file CODICI in versione "innovation intelligence" per integrazione dati
file_risultati = data_dir + '\\' + 'iifvg_codici_filtrato_noncap.csv'
cols_df = pd.read_excel('cols_dict.xlsx', sheet_name='codici',
            usecols = ['nomi_colonne_corretti','ordine_iifvg']).dropna() 
cols_df.sort_values('ordine_iifvg', inplace = True)
cols_to_use = list(cols_df['nomi_colonne_corretti'])
df_codici[cols_to_use].to_csv(      file_risultati, 
                                    sep ='|',   
                                    encoding='utf-8-sig', 
                                    index=False)